In [46]:
import json, re, html, six, pandas as pd, random as rd
from google.cloud import translate_v2 as translate

def translate_text(target, text, char_limit):
    """Translates text into the target language.
    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)
    input_text = re.sub(r"\s+", ' ', result['input'])[:char_limit]
    translated_text = result['translatedText'][:char_limit]
    source_language = result['detectedSourceLanguage']
    print(f"Text: {input_text}")
    print(f"Translation: {translated_text}")
    print(f"Detected source language: {source_language}")
    result = html.unescape(result['translatedText'])
    return result

datafilepath = '/Users/stephenenright-ward/Documents/coding/woodlands/data/bilby-sentiment-data/document-sentiment-data-2020-2021.csv'
data_df = pd.read_csv(datafilepath, nrows=100)
# data_df = pd.read_csv(datafilepath)
data_df.head()

,date,title,text,line,url,sentiment_pos,sentiment_neg,sentiment,energy,materials,...,importance,importance1_score,importance2_score,importance3_score,importance4_score,importance5_score,market_positive_score,market_neutral_score,market_negative_score,market_sentiment
0,2021-12-31,江苏省常州市天宁区,郑 晨\n\n\n近年来，江苏省常州市天宁区在推进社会治理体系和治理能力现代化的进程中...,official,http://epaper.shehuiwang.cn/epaper/zgshb/2021/...,0.983990,0.016010,Positive,0.004482,0.004177,...,3,0.435772,0.399325,0.986421,0.263428,0.039170,2.599501,0.475128,-3.530848,Positive
1,2021-12-31,世界百强建筑设计事务所成功落户,近日，梁黄顾建筑师(香港)事务所（下称“梁黄顾事务所”）顺利在广州市白云区完成商事登记，世界...,official,http://epaper.southcn.com/nfdaily/html/2021-12...,0.997473,0.002527,Positive,0.000014,0.000318,...,1,0.967423,0.566602,0.795377,0.203162,0.044066,2.538163,0.068199,-3.068061,Positive
2,2021-12-31,建设大湾区美丽乡村新典范,当前，珠海正全面实施乡村振兴战略，加快推进农业农村现代化。图为珠海斗门排山村。南方日报记者 ...,official,http://epaper.southcn.com/nfdaily/html/2021-12...,0.868409,0.132324,Positive,0.000112,0.000860,...,3,0.855800,0.559339,0.952253,0.177859,0.040398,2.450996,-0.050573,-2.871006,Positive
3,2021-12-31,新世界云耀：照见未来城市的理想居所即将登场,2021年最后一天，羊城的高端置业者在翘首以盼中，终于迎来岭南新世界沉淀和准备了20年的一份...,official,http://epaper.southcn.com/nfdaily/html/2021-12...,0.999027,0.001089,Positive,0.000031,0.000653,...,1,0.978730,0.533393,0.682044,0.143084,0.057736,2.427758,-0.027683,-2.802187,Positive
4,2021-12-30,河西区完成校外培训机构压减任务,河西区认真贯彻落实关于“双减”工作的有关要求，截至12月16日，在审批和监管部门的协同联动下...,official,http://epaper.tianjinwe.com/tjrb/html/2021-12/...,0.903971,0.096029,Positive,0.000009,0.000497,...,3,0.373541,0.344220,0.989339,0.327839,0.041540,0.793224,2.733099,-2.872527,Neutral


In [22]:
limit = 100
TARGET = 'en'
outputfilepath = './small-data.csv'
results = []
char_limit = 100

for i, idx in enumerate(data_df.iloc[:limit].index):
    zh_title = data_df['title'].loc[idx]
    zh_text = data_df['text'].loc[idx]
    print(f'{i}.')
    en_title = translate_text(TARGET, zh_title, char_limit)
    en_text = translate_text(TARGET, zh_text, char_limit)
    print('')
    data_dct = {'en_text': en_text, 'en_title': en_title}
    results.append(data_dct)

0.
Text: 江苏省常州市天宁区
Translation: Tianning District, Changzhou City, Jiangsu Province
Detected source language: zh-CN
Text: 郑 晨 近年来，江苏省常州市天宁区在推进社会治理体系和治理能力现代化的进程中，坚持党建引领，坚持机制创新，通过“三引三善”，不断提高社会治理能力，使人民群众的获得感、幸福感、安全感不断提升。 厘清社区
Translation: Zheng Chen In recent years, Tianning District, Changzhou City, Jiangsu Province, in the process of p
Detected source language: zh-CN

1.
Text: 世界百强建筑设计事务所成功落户
Translation: The world&#39;s top 100 architectural design firms successfully settled
Detected source language: zh-CN
Text: 近日，梁黄顾建筑师(香港)事务所（下称“梁黄顾事务所”）顺利在广州市白云区完成商事登记，世界百强建筑设计事务所落户白云区永平街。 据了解，这是2020年11月《广东省住房和城乡建设厅关于香港工程建设咨
Translation: Recently, Liang Huang Gu Architects (Hong Kong) Office (hereinafter referred to as &quot;Liang Huang
Detected source language: zh-CN

2.
Text: 建设大湾区美丽乡村新典范
Translation: Building a new model of beautiful countryside in the Greater Bay Area
Detected source language: zh-CN
Text: 当前，珠海正全面实施乡村振兴战略，加快推进农业农村现代化。图为珠海斗门排山村。南方日报记者 关铭荣 摄 让乡村成为生态宜居的美丽家园，让居民望得见山、看得见水、记得住乡愁，是实现乡村

In [19]:
# len(results)
samples = rd.sample(results, 5)

for sample in samples:
    title = sample["en_title"]
    text = sample["en_text"]
    text = text[:100]
    print(f'TITLE: {title}')
    print(f'TEXT: {text}')
    print('')
    

TITLE: Hexi District completed the task of reducing off-campus training institutions
TEXT: Hexi District has conscientiously implemented the relevant requirements on the "double reduction" wo

TITLE: Tianning District, Changzhou City, Jiangsu Province
TEXT: Zheng Chen In recent years, Tianning District, Changzhou City, Jiangsu Province, in the process of p

TITLE: New World Yunyao: The ideal home that sees the city of the future is coming soon
TEXT: On the last day of 2021, high-end property buyers in Yangcheng are eagerly awaiting the arrival of a

TITLE: The world's top 100 architectural design firms successfully settled
TEXT: Recently, Liang Huang Gu Architects (Hong Kong) Office (hereinafter referred to as "Liang Huang Gu O

TITLE: Building a new model of beautiful countryside in the Greater Bay Area
TEXT: At present, Zhuhai is fully implementing the rural revitalization strategy and accelerating the mode



In [27]:
#new_data_df = pd.concat([data_df, pd.DataFrame(results)], axis=1)
en_title_column = pd.Series([data_dct['en_title'] for data_dct in results])
en_text_column = pd.Series([data_dct['en_text'] for data_dct in results])

In [32]:
en_title_column.name = 'en_title'
en_text_column.name = 'en_text'

In [33]:
new_data_df = pd.concat([data_df, en_title_column, en_text_column], axis=1)
new_data_df.head()

,date,title,text,line,url,sentiment_pos,sentiment_neg,sentiment,energy,materials,...,importance2_score,importance3_score,importance4_score,importance5_score,market_positive_score,market_neutral_score,market_negative_score,market_sentiment,en_title,en_text
0,2021-12-31,江苏省常州市天宁区,郑 晨\n\n\n近年来，江苏省常州市天宁区在推进社会治理体系和治理能力现代化的进程中...,official,http://epaper.shehuiwang.cn/epaper/zgshb/2021/...,0.983990,0.016010,Positive,0.004482,0.004177,...,0.399325,0.986421,0.263428,0.039170,2.599501,0.475128,-3.530848,Positive,"Tianning District, Changzhou City, Jiangsu Pro...","Zheng Chen In recent years, Tianning District,..."
1,2021-12-31,世界百强建筑设计事务所成功落户,近日，梁黄顾建筑师(香港)事务所（下称“梁黄顾事务所”）顺利在广州市白云区完成商事登记，世界...,official,http://epaper.southcn.com/nfdaily/html/2021-12...,0.997473,0.002527,Positive,0.000014,0.000318,...,0.566602,0.795377,0.203162,0.044066,2.538163,0.068199,-3.068061,Positive,The world's top 100 architectural design firms...,"Recently, Liang Huang Gu Architects (Hong Kong..."
2,2021-12-31,建设大湾区美丽乡村新典范,当前，珠海正全面实施乡村振兴战略，加快推进农业农村现代化。图为珠海斗门排山村。南方日报记者 ...,official,http://epaper.southcn.com/nfdaily/html/2021-12...,0.868409,0.132324,Positive,0.000112,0.000860,...,0.559339,0.952253,0.177859,0.040398,2.450996,-0.050573,-2.871006,Positive,Building a new model of beautiful countryside ...,"At present, Zhuhai is fully implementing the r..."
3,2021-12-31,新世界云耀：照见未来城市的理想居所即将登场,2021年最后一天，羊城的高端置业者在翘首以盼中，终于迎来岭南新世界沉淀和准备了20年的一份...,official,http://epaper.southcn.com/nfdaily/html/2021-12...,0.999027,0.001089,Positive,0.000031,0.000653,...,0.533393,0.682044,0.143084,0.057736,2.427758,-0.027683,-2.802187,Positive,New World Yunyao: The ideal home that sees the...,"On the last day of 2021, high-end property buy..."
4,2021-12-30,河西区完成校外培训机构压减任务,河西区认真贯彻落实关于“双减”工作的有关要求，截至12月16日，在审批和监管部门的协同联动下...,official,http://epaper.tianjinwe.com/tjrb/html/2021-12/...,0.903971,0.096029,Positive,0.000009,0.000497,...,0.344220,0.989339,0.327839,0.041540,0.793224,2.733099,-2.872527,Neutral,Hexi District completed the task of reducing o...,Hexi District has conscientiously implemented ...


In [48]:
json_data = [row.to_dict() for _, row in new_data_df.iterrows()]
len(json_data)

100

In [49]:
outputfilepath = './small-data.json'

with open(outputfilepath, 'w') as f:
    json.dump(json_data, f)

In [50]:
with open(outputfilepath, 'r') as f:
    new_json_data = json.load(f)

In [51]:
new_json_data[:3]

[{'date': '2021-12-31',
  'title': '江苏省常州市天宁区',
  'text': '郑 \xa0\xa0\xa0晨\n\n\n近年来，江苏省常州市天宁区在推进社会治理体系和治理能力现代化的进程中，坚持党建引领，坚持机制创新，通过“三引三善”，不断提高社会治理能力，使人民群众的获得感、幸福感、安全感不断提升。\n\n\n厘清社区职能，引领社工充分成长，夯实“善治之基”\n\n\n为有效破解基层治理难题，天宁区把党的政治优势转化为社会治理优势，高规格建立区委社区工作委员会，涵盖20个部门，形成了党委、政府联动推进社区建设的“双引擎”，乡镇（街道）设社工委办公室，乡镇（街道）党（工）委副书记为主抓领导，分管领导专职负责日常工作，社区党组织书记全面领导和统筹，实现党组织设置与社区治理单元相契合。实行村社结对共建，推进城乡互助、资源共享、优势互补、协调发展。\n\n\n天宁区在常州市率先开展社区治理创新试点工作，通过厘清政务、居务关系，进一步优化资源配置，减轻社区行政负担，提升为民服务效能。首批试点选取青龙、红梅、雕庄三个街道，在前期调研基础上，结合各自区位、资源特色，明确改革方向。\n\n\n青龙街道率先探索“大社区”模式，成立横塘社区综合服务周边小区，并梳理、分开政务和居务服务事项，集中设立多元服务中心，同步推行全要素网格化管理，提升社区服务的专业度和精细度。“大社区”还充分发挥基层党组织“主心骨”作用，以整体运作的形式共抓治理难点、共议重大事项、共享信息资源。\n\n\n红梅街道、雕庄街道探索“政、居务分开+专业社工”模式，社区部分社工转入社会工作服务中心，在专业团队指导下，运用社会工作的理念与方法，专职从事社区服务工作，提升社区服务专业化水平。\n\n\n为夯实“善治之基”，天宁区持续培育社工领军人才，建立与高校、专业机构合作机制，深化社工督导培训项目，计划继续培养约20名社工督导师，并通过“社校共建”、项目督导等，进一步发挥社工督导的引领作用。\n\n\n通过社区参与，引进资源充分整合，打通“善治之路”\n\n\n针对社区治理创新试点过程中发现的居民自治能力不足、人际关系疏离等问题，天宁区计划用三年时间，推动社会组织参与式陪伴、专业社工对居民骨干进行能力建设、社区整合资源支持居民组织化参与社区事务，让居民熟悉起来、组织起来、参与进来